In [48]:
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt

In [28]:
# Path to your service account key file
key_path = "/home/ollie/.dbt/gcp_crane_service_account.json"

# Explicitly create the client using the key file
client = bigquery.Client.from_service_account_json(key_path)

sql_query = """
SELECT *
FROM `raw.syn_fraud_transactions`
"""

df = client.query(sql_query).to_dataframe()

/home/ollie/.cache/pypoetry/virtualenvs/crane-E909CCJ5-py3.12/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [3]:
#df.to_pickle('../raw_transactions.pkl')
df = pd.read_pickle('../raw_transactions.pkl')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1361088 entries, 0 to 1361087
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   step            1361088 non-null  Int64  
 1   type            1361088 non-null  object 
 2   amount          1361088 non-null  float64
 3   nameOrig        1361088 non-null  object 
 4   oldbalanceOrg   1361088 non-null  float64
 5   newbalanceOrig  1361088 non-null  float64
 6   nameDest        1361088 non-null  object 
 7   oldbalanceDest  1361088 non-null  float64
 8   newbalanceDest  1361088 non-null  float64
 9   isFraud         1361088 non-null  Int64  
 10  isFlaggedFraud  1361088 non-null  Int64  
dtypes: Int64(3), float64(5), object(3)
memory usage: 118.1+ MB


In [5]:
df

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,CASH_IN,35902.49,C839771540,371688.15,407590.65,C2001112025,49003.30,0.00,0,0
1,1,CASH_IN,65912.95,C180316302,640544.28,706457.23,C1330106945,104198.26,24044.18,0,0
2,1,CASH_IN,418688.27,C1756819670,1888091.55,2306779.82,C401424608,956455.03,1178808.14,0,0
3,1,CASH_IN,151862.38,C178604517,2617803.34,2769665.72,C920011586,158858.48,0.00,0,0
4,1,CASH_IN,228710.05,C57624756,2769665.72,2998375.78,C1297685781,743792.14,16997.22,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1361083,138,TRANSFER,1326583.07,C189359242,0.00,0.00,C793302743,6114311.66,7440894.72,0,0
1361084,138,TRANSFER,651510.11,C1651586939,0.00,0.00,C1447542600,4064832.45,4716342.56,0,0
1361085,138,TRANSFER,773569.58,C1270408261,0.00,0.00,C1109945960,2142176.47,2915746.05,0,0
1361086,138,TRANSFER,269645.68,C1261311336,0.00,0.00,C1804726443,280699.28,550344.96,0,0


In [33]:
df.isna().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [27]:
print("TRANSACTION TYPE ANALYSIS")
type_counts = df['type'].value_counts()
type_percentages = df['type'].value_counts(normalize=True) * 100

for i in range(len(type_counts)):
    print(f"{type_counts.index[i]:<10} {type_counts[i]:>10.2f}    {type_percentages[i]:>6.2f}%")

TRANSACTION TYPE ANALYSIS
CASH_OUT    486218.00     35.72%
PAYMENT     455066.00     33.43%
CASH_IN     297771.00     21.88%
TRANSFER    112999.00      8.30%
DEBIT         9034.00      0.66%


/tmp/ipykernel_4764/932999481.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{type_counts.index[i]:<10} {type_counts[i]:>10.2f}    {type_percentages[i]:>6.2f}%")


In [36]:
df[df.duplicated()]

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud


In [46]:
# See if we have both sides of some transactions? No.
pd.merge(
    df[df['type'] == 'CASH_IN'], 
    df[df['type'] == 'CASH_OUT'], 
    left_on=['amount', 'nameOrig'], 
    right_on=['amount', 'nameDest'], 
    how='inner'
)


,step_x,type_x,amount,nameOrig_x,oldbalanceOrg_x,newbalanceOrig_x,nameDest_x,oldbalanceDest_x,newbalanceDest_x,isFraud_x,...,step_y,type_y,nameOrig_y,oldbalanceOrg_y,newbalanceOrig_y,nameDest_y,oldbalanceDest_y,newbalanceDest_y,isFraud_y,isFlaggedFraud_y


In [57]:
# Seemingly not used.
df.isFlaggedFraud.value_counts()

isFlaggedFraud
0    1361088
Name: count, dtype: Int64

In [60]:
df.isFraud.value_counts() #.sum() same total as isFlaggedFraud, no nans.

isFraud
0    1359520
1       1568
Name: count, dtype: Int64